In [1]:
%env KIF_DEBUG=
%env KIF_RESOLVE_ENTITIES=

env: KIF_DEBUG=
env: KIF_RESOLVE_ENTITIES=


In [2]:
import time

# Async

In [3]:
from kif_lib import *
from kif_lib.vocabulary import pc, wd

## The async Store API

The Store API exposes async versions of some methods:

|sync|async|description|
|-|-|-|
|**contains**(stmt)|**acontains**(stmt)|tests whether `stmt` occurs in store|
|**ask**(s,p,v)|**aask**(s,p,v)|tests whehter any statement matches `s,p,v`|
|**count**(s,p,v)|**acount**(s,p,v)|counts the number of statements matching `s,p,v`|
|**filter**(s,p,v)|**afilter**(s,p,v)|fetches statements matching `s,p,v`|
|**filter_annotated**(s,p,v)|**afilter_annotated**(s,p,v)|fetches annotated statements matching `s,p,v`|

The async versions behave similarly to their sync counterparts. The difference is that they can only be called within an event loop and their results need to be "awaited".

(Note that Jupyter cells are awalys run within an event loop.)

For example,

In [4]:
kb = Store('wdqs')

In [5]:
await kb.acontains(wd.shares_border_with(wd.Brazil, wd.Argentina))

True

In [6]:
await kb.aask(subject=wd.Brazil, property=wd.shares_border_with, value=wd.United_States_of_America)

False

In [7]:
await kb.acount(subject=wd.Brazil, property=wd.shares_border_with)

11

The `afilter()` method returns an [AsyncIterator](https://docs.python.org/3/glossary.html#term-asynchronous-iterator):

In [8]:
it = kb.afilter(wd.Brazil, wd.shares_border_with)

In [9]:
type(it)

async_generator

To consume an async iterator, we must use an async for-loop:

In [10]:
async for stmt in it:
    display(stmt)

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Peru](http://www.wikidata.org/entity/Q419))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Venezuela](http://www.wikidata.org/entity/Q717))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Suriname](http://www.wikidata.org/entity/Q730))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Guyana](http://www.wikidata.org/entity/Q734))))

...

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Colombia](http://www.wikidata.org/entity/Q739))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Bolivia](http://www.wikidata.org/entity/Q750))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [French Guiana](http://www.wikidata.org/entity/Q3769))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Uruguay](http://www.wikidata.org/entity/Q77))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [France](http://www.wikidata.org/entity/Q142))))

## Lookahead

One of the advantages of the async Store API is that it opens up opportunities for parallelization.

The `Store.lookahead` option determines the number of "parallel" (non-blocking) requests done by `afilter()` when advancing its results.

A `Store.lookahead` value of *n* (the default *n* is 2) means that  `afilter()` will fetch *n* consecutive pages in parallel at a time.

For instance, if we set `Store.lookahead` to 4 and `Store.page_size` to 2, then `afilter()` will advance its results using 4 parallel requests of size 2. 

In [11]:
kb.lookahead = 4
kb.page_size = 2

To see the effects of this, let's enable logging:

In [12]:
import logging
logging.basicConfig(level=logging.INFO)

Note that `afilter()` generates two parallel requests whenever new results are needed:

In [13]:
dt = time.time()
async for stmt in kb.afilter(wd.Brazil, wd.shares_border_with):
    display(stmt)
print(f'*** TOOK {time.time() - dt:0.2f}s ***')

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Peru](http://www.wikidata.org/entity/Q419))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Venezuela](http://www.wikidata.org/entity/Q717))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Suriname](http://www.wikidata.org/entity/Q730))))

...

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [French Guiana](http://www.wikidata.org/entity/Q3769))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Uruguay](http://www.wikidata.org/entity/Q77))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [France](http://www.wikidata.org/entity/Q142))))

*** TOOK 0.73s ***


Contrast this with the synchronous version:

In [14]:
dt = time.time()
for stmt in kb.filter(wd.Brazil, wd.shares_border_with):
    display(stmt)
print(f'*** TOOK {time.time() - dt:0.2f}s ***')

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Peru](http://www.wikidata.org/entity/Q419))))

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Venezuela](http://www.wikidata.org/entity/Q717))))

...

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Uruguay](http://www.wikidata.org/entity/Q77))))

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [France](http://www.wikidata.org/entity/Q142))))

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


*** TOOK 1.29s ***


## Asyncronous Mixer calls

The async version of the Mixer Store's `filter()` calls are evaluated parallel.

That is, the filter is applied in parallel to the child stores.

For example,

In [15]:
mx = Store('mixer', (Store('wdqs'), Store('pubchem-sparql')))
async for stmt in mx.afilter(property=wd.canonical_SMILES, value='C1=CC=CC=C1'):
    display(stmt)

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qlever.cs.uni-freiburg.de/api/pubchem "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qlever.cs.uni-freiburg.de/api/pubchem "HTTP/1.1 200 OK"


(**Statement** (**Item** [benzene](http://www.wikidata.org/entity/Q2270)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID101103333](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID101103333)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [wd:Q1101314](http://www.wikidata.org/entity/Q1101314)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID102602330](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID102602330)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

...

(**Statement** (**Item** [pc:CID57190391](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID57190391)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID57264323](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID57264323)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID71308903](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID71308903)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID71309208](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID71309208)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID71601](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID71601)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

## Evaluating filters in parallel

Sometimes one need to evaluate multiple filters over the same store.

This can be done convinently using the methods `Store.mix` and `Store.amix`.

The advantage of the async version, `Store.amix`, is that is performes the evaluation in parallel.

For example, consider the following store:

In [16]:
kb = Store('wdqs')
kb.lookahead = 4
kb.page_size = 2

Let us now use `Store.amix` to evaluate mutiple filters in parallel:

In [17]:
it = kb.amix(
    Filter(wd.Brazil, wd.shares_border_with),
    Filter(wd.Argentina, wd.shares_border_with),
    Filter(wd.Paraguay, wd.shares_border_with))

dt = time.time()
async for stmt in it:
    display(stmt)
print(f'*** TOOK {time.time() - dt:0.2f}s ***')

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Argentina](http://www.wikidata.org/entity/Q414)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Chile](http://www.wikidata.org/entity/Q298))))

(**Statement** (**Item** [Paraguay](http://www.wikidata.org/entity/Q733)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Peru](http://www.wikidata.org/entity/Q419))))

...

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"


(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [French Guiana](http://www.wikidata.org/entity/Q3769))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Uruguay](http://www.wikidata.org/entity/Q77))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [France](http://www.wikidata.org/entity/Q142))))

*** TOOK 1.02s ***


Contrast this with the synchronous version:

In [18]:
it = kb.mix(
    Filter(wd.Brazil, wd.shares_border_with),
    Filter(wd.Argentina, wd.shares_border_with),
    Filter(wd.Paraguay, wd.shares_border_with))
dt = time.time()
display(*it)

print(f'*** TOOK {time.time() - dt:0.2f}s ***')

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP 

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Argentina](http://www.wikidata.org/entity/Q414)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Chile](http://www.wikidata.org/entity/Q298))))

(**Statement** (**Item** [Paraguay](http://www.wikidata.org/entity/Q733)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Argentina](http://www.wikidata.org/entity/Q414))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Peru](http://www.wikidata.org/entity/Q419))))

...

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Bolivia](http://www.wikidata.org/entity/Q750))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [French Guiana](http://www.wikidata.org/entity/Q3769))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [Uruguay](http://www.wikidata.org/entity/Q77))))

(**Statement** (**Item** [Brazil](http://www.wikidata.org/entity/Q155)) (**ValueSnak** (**Property** [shares border with](http://www.wikidata.org/entity/P47)) (**Item** [France](http://www.wikidata.org/entity/Q142))))

*** TOOK 3.63s ***


Finally, note that `Store.mix` and `Store.amix` also work with mixer stores:

In [22]:
it = mx.amix(
    Filter(property=wd.canonical_SMILES, value='C1=CC=CC=C1'),
    Filter(property=wd.InChIKey, value='BSYNRYMUTXBXSQ-UHFFFAOYSA-N'),
    Filter(property=wd.chemical_formula, value='H2O'),
    limit = 10)
async for stmt in it:
    display(stmt)

INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://query.wikidata.org/sparql "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qlever.cs.uni-freiburg.de/api/pubchem "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qlever.cs.uni-freiburg.de/api/pubchem "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://qlever.cs.uni-freiburg.de/api/pubchem "HTTP/1.1 200 OK"


(**Statement** (**Item** [benzene](http://www.wikidata.org/entity/Q2270)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [wd:Q18216](http://www.wikidata.org/entity/Q18216)) (**ValueSnak** (**Property** [InChIKey](http://www.wikidata.org/entity/P235)) "BSYNRYMUTXBXSQ-UHFFFAOYSA-N"))

(**Statement** (**Item** [pc:CID10129877](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID10129877)) (**ValueSnak** (**Property** [chemical formula](http://www.wikidata.org/entity/P274)) "H2O"))

(**Statement** (**Item** [pc:CID101103333](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID101103333)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

...

(**Statement** (**Item** [pc:CID10197601](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID10197601)) (**ValueSnak** (**Property** [chemical formula](http://www.wikidata.org/entity/P274)) "H2O"))

(**Statement** (**Item** [wd:Q1101314](http://www.wikidata.org/entity/Q1101314)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID10214376](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID10214376)) (**ValueSnak** (**Property** [chemical formula](http://www.wikidata.org/entity/P274)) "H2O"))

(**Statement** (**Item** [pc:CID102602330](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID102602330)) (**ValueSnak** (**Property** [canonical SMILES](http://www.wikidata.org/entity/P233)) "C1=CC=CC=C1"))

(**Statement** (**Item** [pc:CID104752](http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID104752)) (**ValueSnak** (**Property** [chemical formula](http://www.wikidata.org/entity/P274)) "H2O"))